In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv("../data/raw/train.csv")
store = pd.read_csv("../data/raw/store.csv")

train['Date'] = pd.to_datetime(train['Date'])


C:\Users\rupes\AppData\Local\Temp\ipykernel_15028\1982636090.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/raw/train.csv")


In [3]:
train.head()
train.info()
store.head()
store.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  int64         
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  int64         
 6   Promo          1017209 non-null  int64         
 7   StateHoliday   1017209 non-null  object        
 8   SchoolHoliday  1017209 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 69.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store      

In [4]:
train.shape, store.shape
train['Date'].min(), train['Date'].max()
train['Store'].nunique()


1115

In [5]:
train['Sales'].describe()
(train['Sales'] == 0).sum()


np.int64(172871)

In [6]:
train.groupby('Open')['Sales'].describe()


,count,mean,std,min,25%,50%,75%,max
Open,,,,,,,,
0,172817.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0
1,844392.0,6955.514291,3104.21468,0.0,4859.0,6369.0,8360.0,41551.0


In [7]:
merged_preview = train.merge(store, on='Store', how='left')
merged_preview.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [8]:
merged_preview.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [9]:
df = merged_preview.copy()


In [10]:
df.isna().sum().sort_values(ascending=False)


Promo2SinceWeek              508031
PromoInterval                508031
Promo2SinceYear              508031
CompetitionOpenSinceYear     323348
CompetitionOpenSinceMonth    323348
CompetitionDistance            2642
DayOfWeek                         0
Store                             0
Date                              0
Sales                             0
StoreType                         0
SchoolHoliday                     0
StateHoliday                      0
Promo                             0
Open                              0
Customers                         0
Assortment                        0
Promo2                            0
dtype: int64

In [11]:
df[df['Open'] == 0]['Sales'].unique()


array([0])

In [12]:
df = df[df['Open'] == 1]


In [13]:
df['CompetitionDistance'].isna().mean()


np.float64(0.0025888449914257834)

In [14]:
df['CompetitionDistance'] = df.groupby('StoreType')['CompetitionDistance'] \
                               .transform(lambda x: x.fillna(x.median()))


In [15]:
df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(0)
df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(0)

df['CompetitionAge'] = (
    (df['Date'].dt.year - df['CompetitionOpenSinceYear']) * 12 +
    (df['Date'].dt.month - df['CompetitionOpenSinceMonth'])
)

df['CompetitionAge'] = df['CompetitionAge'].clip(lower=0)


In [16]:
df['Promo2SinceYear'] = df['Promo2SinceYear'].fillna(0)
df['Promo2SinceWeek'] = df['Promo2SinceWeek'].fillna(0)
df['PromoInterval'] = df['PromoInterval'].fillna('')

df['IsPromo2Active'] = np.where(
    (df['Promo2'] == 1) &
    (df['Date'].dt.year > df['Promo2SinceYear']),
    1, 0
)


In [17]:
df['StateHoliday'] = df['StateHoliday'].replace({'0': 'None'})


In [18]:
df = df.sort_values(['Store', 'Date'])


In [19]:
df.to_csv("../data/processed/rossmann_cleaned.csv", index=False)


In [20]:
demo = pd.read_csv("../data/processed/rossmann_cleaned.csv")

In [21]:
demo.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionAge,IsPromo2Active
0,1,3,2013-01-02,5530,668,1,0,NaN,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,NaN,52.0,0
1,1,4,2013-01-03,4327,578,1,0,NaN,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,NaN,52.0,0
2,1,5,2013-01-04,4486,619,1,0,NaN,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,NaN,52.0,0
3,1,6,2013-01-05,4997,635,1,0,NaN,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,NaN,52.0,0
4,1,1,2013-01-07,7176,785,1,1,NaN,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,NaN,52.0,0


In [22]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844392 entries, 0 to 844391
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Store                      844392 non-null  int64  
 1   DayOfWeek                  844392 non-null  int64  
 2   Date                       844392 non-null  object 
 3   Sales                      844392 non-null  int64  
 4   Customers                  844392 non-null  int64  
 5   Open                       844392 non-null  int64  
 6   Promo                      844392 non-null  int64  
 7   StateHoliday               113050 non-null  object 
 8   SchoolHoliday              844392 non-null  int64  
 9   StoreType                  844392 non-null  object 
 10  Assortment                 844392 non-null  object 
 11  CompetitionDistance        844392 non-null  float64
 12  CompetitionOpenSinceMonth  844392 non-null  float64
 13  CompetitionOpenSinceYear   84